In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

from shared import read_dataset, plot_results,scatter_results, evaluate_price_predictions, mean_abs_error

2023-11-05 14:55:07.044366: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 14:55:07.071318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 14:55:07.071354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 14:55:07.071375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 14:55:07.076867: I tensorflow/core/platform/cpu_feature_g

In [2]:
start = "2010-01-01"
end = "2020-01-01"
target_column_name = 'Adj Close'
model_path = '../models/lstm_model/price_generator.h5'

GS = read_dataset('../data/GS.csv', start, end)

GS['Close_Diff'] = GS['Close'].diff()
GS.dropna(inplace=True) # initially NAN for moving averages

cols = [target_column_name] + [ col for col in GS if col != target_column_name]
target_column = list(GS.columns).index(target_column_name)
data = GS[cols]

# Define feature array and target array to train the model.
data_array = np.array(data.values)
target_array = np.array(data[target_column_name].values).reshape(-1, 1)

# Normalize the data
scaler_data = MinMaxScaler()
scaler_data.fit(data_array)
data_array = scaler_data.transform(data_array)

scaler_target = MinMaxScaler()
scaler_target.fit(target_array)
target_array = scaler_target.transform(target_array)

# Split the data
train_size = int(len(data_array) * 0.50)
evaluation_size = int(len(data_array) * 0.20) # no evaluation

# Define data sequences
def create_sequences(data, target, seq_length):
    sequence_data = []
    sequence_target = []
    for i in range(seq_length, len(data)):
        sequence_data.append(data[i-seq_length:i])
        sequence_target.append(target[i])
    return np.array(sequence_data), np.array(sequence_target)

SEQUENCE_LENGTH = 30
data_sequences, target_sequences = create_sequences(data_array, target_array, SEQUENCE_LENGTH)

# Shuffle the data
shuffle_idxs = np.random.permutation(len(data_sequences))
revert_idxs = np.argsort(shuffle_idxs)

data_sequences = data_sequences[shuffle_idxs]
target_sequences = target_sequences[shuffle_idxs]

# Split the data into Train | Evaluation | Test datasets
train_data, eval_data, test_data = data_sequences[:train_size], data_sequences[train_size:train_size+evaluation_size], data_sequences[train_size+evaluation_size:]
train_target, eval_target, test_target = target_sequences[:train_size], target_sequences[train_size:train_size+evaluation_size], target_sequences[train_size+evaluation_size:]

train_data.shape

(1248, 30, 21)

In [ ]:
def build_model_just_autoencoded(input_shape):
    """LSTM model that takes encoded input"""
    model = keras.Sequential()
    model.add(layers.Dense(64, input_shape=input_shape, activation='gelu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1))
    return model

def create_autoencoder(input_shape, encoding_dim):
    input_seq = layers.Input(shape=input_shape)

    # Encoder
    encoded = layers.LSTM(encoding_dim, activation='relu', return_sequences=True)(input_seq)
    encoded = layers.LSTM(int(encoding_dim/2), activation='relu', return_sequences=False)(encoded)
    encoded = layers.BatchNormalization()(encoded)
    encoded = layers.Dropout(0.3) (encoded)
    repeadted_encoding = layers.RepeatVector(input_shape[0])(encoded)  # repeat the encoding ro reshape it back into a sequence

    # Decoder
    decoded = layers.LSTM(int(encoding_dim/2), return_sequences=True, activation='relu')(repeadted_encoding)
    decoded = layers.LSTM(encoding_dim, return_sequences=True, activation='relu')(decoded)
    decoded = layers.BatchNormalization()(decoded)
    decoded = layers.Dropout(0.3)(decoded)
    decoded = layers.TimeDistributed(layers.Dense(input_shape[1]))(decoded)

    autoencoder = keras.Model(input_seq, decoded)
    encoder = keras.Model(input_seq, encoded)

    return autoencoder, encoder

In [ ]:
# Size of encoding layer
encoding_dim = 1024
autoencoder_model_path = '../models/autoencoder/autoencoder.h5'

input_shape = (train_data.shape[1], train_data.shape[2])
autoencoder, encoder = create_autoencoder(input_shape, encoding_dim)
# autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# autoencoder_checkpoint = keras.callbacks.ModelCheckpoint(
#     autoencoder_model_path,
#     monitor='val_loss',
#     verbose=1,
#     save_best_only=True,
#     mode='min'
# )
# autoencoder.fit(train_data, train_data,
#                 epochs=200,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(eval_data, eval_data),
#                 verbose=2,
#                 callbacks=[autoencoder_checkpoint])

autoencoder = load_model(autoencoder_model_path)

In [ ]:
encoded_train_data = encoder.predict(train_data)
encoded_eval_data = encoder.predict(eval_data)
encoded_test_data = encoder.predict(test_data)

prediction_model_path = '../models/autoencoder/predictor.h5'

encoded_input_shape = (encoded_train_data.shape[1],)
prediction_model = build_model_just_autoencoded(encoded_input_shape)
# prediction_model.compile(optimizer='adam', loss='mean_absolute_error')

# predictor_checkpoint = keras.callbacks.ModelCheckpoint(
#     prediction_model_path,
#     monitor='val_loss',
#     verbose=1,
#     save_best_only=True,
#     mode='min'
# )

# prediction_model.fit(
#                     encoded_train_data,
#                     train_target,
#                     epochs=1000,
#                     batch_size=256,
#                     shuffle=True,
#                     verbose=2,
#                     validation_data=(encoded_eval_data, eval_target),
#                     callbacks=[predictor_checkpoint],
#                     )

prediction_model = load_model(prediction_model_path)

In [ ]:
price_predicted_array = prediction_model.predict(encoded_train_data)
price_predicted_array = scaler_target.inverse_transform(price_predicted_array)  # Denormalize predictions

price_actual_array = scaler_target.inverse_transform(train_target.reshape(-1, 1)).flatten()  # Denormalize actual values

# Plot the results
plot_results(price_actual_array, price_predicted_array, target_column_name, title="Train Data")

In [ ]:
price_predicted_array = prediction_model.predict(encoded_eval_data)
price_predicted_array = scaler_target.inverse_transform(price_predicted_array)  # Denormalize predictions

price_actual_array = scaler_target.inverse_transform(eval_target.reshape(-1, 1)).flatten()  # Denormalize actual values

# Plot the results
plot_results(price_actual_array, price_predicted_array, target_column_name, title='Evaluation Data')


In [ ]:
price_predicted_array = prediction_model.predict(encoded_test_data)
price_predicted_array = scaler_target.inverse_transform(price_predicted_array)  # Denormalize predictions

price_actual_array = scaler_target.inverse_transform(test_target.reshape(-1, 1)).flatten()  # Denormalize actual values

# Plot the results
plot_results(price_actual_array, price_predicted_array, target_column_name, title='Test Data')

The results obtained aren't good enough. By deleting the evaluation part they get better, but my deduction is that it's just thanks to overfitting and not thanks 

In [ ]:
price_predicted_array = prediction_model.predict(encoded_test_data)
price_predicted_array = scaler_target.inverse_transform(price_predicted_array)
price_actual_array = scaler_target.inverse_transform(test_target.reshape(-1, 1)).flatten()

evaluate_price_predictions(price_predicted_array, price_actual_array)

In [ ]:
recomposed_target = np.concatenate([train_target, eval_target, test_target])
recomposed_target = scaler_target.inverse_transform(recomposed_target[revert_idxs])

predicted_target = np.concatenate([train_target, eval_target, prediction_model.predict(encoded_test_data)])
predicted_target = scaler_target.inverse_transform(predicted_target[revert_idxs])

scatter_results(recomposed_target, predicted_target, target_column_name, title='All together')


# Status: Work in Progress
- non sta funzionando: neanche con shuffle data.